In [ ]:
from joinboost_dp import *

In [ ]:
sales = pd.read_csv("data/train.csv")

In [ ]:
cons = dict()
for i in range(2):
    cons[i] = duckdb.connect(database=':memory:', check_same_thread=False)
    print("reading tables into duckdb...")
    cons[i].execute("CREATE OR REPLACE TABLE holidays AS SELECT * FROM 'data/holidays.csv';")
    cons[i].execute("CREATE OR REPLACE TABLE oil AS SELECT * FROM 'data/oil.csv';")
    cons[i].execute("CREATE OR REPLACE TABLE transactions AS SELECT * FROM 'data/transactions.csv';")
    cons[i].execute("CREATE OR REPLACE TABLE stores AS SELECT * FROM 'data/stores.csv';")
    cons[i].execute("CREATE OR REPLACE TABLE items AS SELECT * FROM 'data/items.csv';")
#     cons[i].execute("CREATE OR REPLACE TABLE test AS SELECT * FROM 'data/test.csv';")
    print("done")

In [ ]:
jg = joinGraph("favorita", cons, log=False, max_leaves = 8,  learning_rate=0.1, target_variable ="Y", fact_pandas=True, fact_pandas_join=True)
jg.add_table("sales", [], [], fact=True, fact_table=sales)
jg.add_table("holidays", ["htype", "locale", "locale_name", "transferred","f2"], [2,2,2,2,2])
jg.add_table("oil", ["dcoilwtico","f3"], [2,2])
jg.add_table("transactions", ["transactions","f5"], [2,2])
jg.add_table("stores", ["city","state","stype","cluster","f4"], [2,2,2,2,2])
jg.add_table("items", ["family","class","perishable","f1"], [2,2,2,2])
jg.join("sales", "items", ["item_nbr"], ["item_nbr"])
jg.join("sales", "transactions", ["tid"], ["tid"])
jg.join("transactions", "stores", ["store_nbr"], ["store_nbr"])
jg.join("transactions", "holidays", ["date"], ["date"])
jg.join("holidays", "oil", ["date"], ["date"])

In [ ]:
jg.create_dummy_model(replace=True)
for i in cons:
    cons[i].register('sales', sales)
total = 0
for i in range(100):
    jg.create_base_node()
    jg.build_gradient_tree()
    jg.update_error()
    jg.clean_leaves()
    jg.clean_table()
    print(str(i) + ", " + str(jg.totaltime))